# RDFIA : Notebook Transfer Learning

## Fonctions utilitaires

In [1]:
import matplotlib.pyplot as plt
plt.ion()
import numpy as np

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, keep_all=False):
        self.reset()
        self.data = None
        if keep_all:
            self.data = []

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        if self.data is not None:
            self.data.append(val)
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class TrainLossPlot(object):
    def __init__(self):
        self.loss_train = []
        self.fig = plt.figure()

    def update(self, loss_train):
        self.loss_train.append(loss_train)

    def plot(self):
        plt.figure(self.fig.number)
        plt.clf()
        plt.plot(np.array(self.loss_train))
        plt.title("Train loss / batch")
        plt.xlabel("Batch")
        plt.ylabel("Loss")
        plt.show()
        plt.draw_all()
        plt.pause(1e-3)

class AccLossPlot(object):
    def __init__(self):
        self.loss_train = []
        self.loss_test = []
        self.acc_train = []
        self.acc_test = []
        self.fig = plt.figure()

    def update(self, loss_train, loss_test, acc_train, acc_test):
        self.loss_train.append(loss_train)
        self.loss_test.append(loss_test)
        self.acc_train.append(acc_train)
        self.acc_test.append(acc_test)
        plt.figure(self.fig.number)
        plt.clf()
        plt.subplot(1,2,1)
        plt.plot(np.array(self.acc_train), label="acc. train")
        plt.plot(np.array(self.acc_test), label="acc. test")
        plt.title("Accuracy / epoch")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.subplot(1,2,2)
        plt.plot(np.array(self.loss_train), label="loss train")
        plt.plot(np.array(self.loss_test), label="loss test")
        plt.title("Loss / epoch")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        plt.draw_all()
        plt.pause(1e-3)


## Imports et constantes

In [2]:
import argparse
import os
import time

from PIL import Image

import numpy as np

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.svm import LinearSVC
from numpy import linalg as LA

torchvision.models.vgg.model_urls["vgg16"] = "http://webia.lip6.fr/~robert/cours/rdfia/vgg16-397923af.pth"
os.environ["TORCH_MODEL_ZOO"] = "/tmp/torch"
PRINT_INTERVAL = 50
CUDA = False

## Récupération du dataset et compose

In [3]:
def get_dataset(batch_size, path):
    """
    Cette fonction charge le dataset et effectue des transformations sur chaqu
    image (listées dans `transform=...`).
    """
    train_dataset = datasets.ImageFolder(path+'/train',
        transform=transforms.Compose([ transforms.Scale((224,224)),
            transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
        ]))
    val_dataset = datasets.ImageFolder(path+'/test',
        transform=transforms.Compose([ transforms.Scale((224,224)),
            transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
        ]))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                        batch_size=batch_size, shuffle=True, pin_memory=CUDA, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                        batch_size=batch_size, shuffle=False, pin_memory=CUDA, num_workers=2)

    return train_loader, val_loader

## Différents modèles pré-entraînés

VGG16 jusqu'à ReLU 7

In [4]:
class VGG16relu7(nn.Module):

    def __init__(self, vgg16):

        super(VGG16relu7, self).__init__()
        #recopier toute la partie convolutionnelle
        self.features = nn.Sequential( *list(vgg16.features.children()))
        # garder une partie du classifieur, -2 pour s’arrêter à relu7
        self.classifier = nn.Sequential( *list(vgg16.classifier.children())[:-2])
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

VGG16 jusqu'à ReLU 5

In [5]:
class VGG16relu5(nn.Module):

    #whitout last maxPooling
    # we can't add the vgg16.classifier cos it takes 25088 = 7*7*512
    def __init__(self, vgg16):

        super(VGG16relu5, self).__init__()
        #recopier toute la partie convolutionnelle
        self.features = nn.Sequential( *list(vgg16.features.children())[:-1])
        # garder une partie du classifieur, -2 pour s’arrêter à relu7
    def forward(self, x):
        x = self.features(x)
        print(x.size())
        x = x.view(x.size(0), -1)
        print(x.size())
        return x
    #size features in this case = 100352

VGG16 jusqu'à ReLU 6

In [6]:
class VGG16relu6(nn.Module):

    #stop in the fist fc
    def __init__(self, vgg16):

        super(VGG16relu6, self).__init__()
        #recopier toute la partie convolutionnelle
        self.features = nn.Sequential( *list(vgg16.features.children()))
        # garder une partie du classifieur, -2 pour s’arrêter à relu7
        self.classifier = nn.Sequential( *list(vgg16.classifier.children())[:-5])

    def forward(self, x):
        x = self.features(x)
        print(x.size())
        x = x.view(x.size(0), -1)
        print(x.size())
        x = self.classifier(x)
        print(x.size())
        return x

AlexNet

In [7]:
class Alexnet(nn.Module):

    def __init__(self, alex):

        super(Alexnet, self).__init__()
        self.features = nn.Sequential( *list(alex.features.children()))
        self.classifier = nn.Sequential( *list(alex.classifier.children())[:-1])
    def forward(self, x):
        print(x.size())
        x = self.features(x)
        print(x.size())
        x = x.view(x.size(0), -1)
        print(x.size())
        x = self.classifier(x)
        print(x.size())
        return x

VGG16 jusqu'à fc15

In [8]:
class VGG16fc15(nn.Module):

    def __init__(self, vgg16):

        super(VGG16fc15, self).__init__()
        #recopier toute la partie convolutionnelle
        self.features = nn.Sequential( *list(vgg16.features.children()))
        # garder une partie du classifieur, -2 pour s’arrêter à relu7
        newliste = list(vgg16.classifier.children())[:-1]
        newliste.append(nn.Linear(4096, 15))
        self.classifier = nn.Sequential( *newliste)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

## Définition d'une epoch

In [9]:
def epoch(data, model, criterion, optimizer=None):

    model.eval() if optimizer is None else model.train()

    # objets pour stocker les moyennes des metriques
    avg_loss = AverageMeter()
    avg_top1_acc = AverageMeter()
    avg_top5_acc = AverageMeter()
    avg_batch_time = AverageMeter()
    global loss_plot

    # on itere sur les batchs du dataset
    tic = time.time()
    for i, (input, target) in enumerate(data):

        if CUDA: # si on fait du GPU, passage en CUDA
            input = input.cuda()
            target = target.cuda()

        # forward
        output = model(Variable(input))
        
        loss = criterion(output, Variable(target))

        # backward si on est en "train"
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # calcul des metriques
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        batch_time = time.time() - tic
        tic = time.time()

        # mise a jour des moyennes
        avg_loss.update(loss.data[0])
        avg_top1_acc.update(prec1[0])
        avg_top5_acc.update(prec5[0])
        avg_batch_time.update(batch_time)
        if optimizer:
            loss_plot.update(avg_loss.val)

        # affichage des infos
        if i % PRINT_INTERVAL == 0:
            print('[{0:s} Batch {1:03d}/{2:03d}]\t'
                  'Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s)\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:5.1f} ({top1.avg:5.1f})\t'
                  'Prec@5 {top5.val:5.1f} ({top5.avg:5.1f})'.format(
                   "EVAL" if optimizer is None else "TRAIN", i, len(data), batch_time=avg_batch_time, loss=avg_loss,
                   top1=avg_top1_acc, top5=avg_top5_acc))
            if optimizer:
                loss_plot.plot()

    # Affichage des infos sur l'epoch
    print('\n===============> Total time {batch_time:d}s\t'
          'Avg loss {loss.avg:.4f}\t'
          'Avg Prec@1 {top1.avg:5.2f} %\t'
          'Avg Prec@5 {top5.avg:5.2f} %\n'.format(
           batch_time=int(avg_batch_time.sum), loss=avg_loss,
           top1=avg_top1_acc, top5=avg_top5_acc))

    return avg_top1_acc, avg_top5_acc, avg_loss

## Entraînement du VGG fc15

In [10]:
def trainVGGfc(vgg16, train, test, max_Iter, step, CUDA=False):
    
    for p in vgg16.parameters():
    	p.requires_grad = False
    model = VGG16fc15(vgg16)
    if CUDA:
        model.cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD([p for p in model.parameters() if p.requires_grad], step)
    # init plots
    plot = AccLossPlot()
    global loss_plot
    loss_plot = TrainLossPlot()

    # On itère sur les epochs
    for i in range(max_Iter):
        print("=================\n=== EPOCH "+str(i+1)+" =====\n=================\n")
        # Phase de train
        top1_acc, avg_top5_acc, loss = epoch(train, model, criterion, optimizer)
        # Phase d'evaluation
        top1_acc_test, top5_acc_test, loss_test = epoch(test, model, criterion)
        # plot
        plot.update(loss.avg, loss_test.avg, top1_acc.avg, top1_acc_test.avg)

## Extraction de features depuis les réseaux

In [11]:
def extract_features(data, model):
    X = []
    y = []
    for i, (input, target) in enumerate(data):
        print(i)
        if i % PRINT_INTERVAL == 0:
            print('Batch {0:03d}/{1:03d}'.format(i, len(data)))
        if CUDA:
            input = Variable(input.type(torch.cuda.FloatTensor), 
                                             requires_grad=False)
        else:
            input = Variable(input.type(torch.FloatTensor), 
                                             requires_grad=False)
        output = model(input).data.cpu().numpy()
        #print(output.shape)
        for o in range(len(output)):
            norme = LA.norm(output[o])
            X.append([float(i)/norme for i in output[o]])
            y.append(target[o])
    return X, y

## Définition du main

In [12]:
def main():
    #print('Instanciation de VGG16')
    #vgg16 = models.vgg16(pretrained=True)
    #squeeznet = torchvision.models.squeezenet1_1(pretrained=True)
    alex = torchvision.models.alexnet(pretrained=True)
    #print(*list(squeeznet.features.children()))
    #print(*list(squeeznet.classifier.children()))
   
#    print('Instanciation de VGG16relu7')
#    model = VGG16relu7(vgg16)
#    model = VGG16relu5(vgg16)
#   model.eval()
    model = Alexnet(alex)
#    model.eval()
    #model.cuda()
#    model = VGG16relu5(vgg16) 
        

    #model.eval()
    if CUDA: # si on fait du GPU, passage en CUDA
        model = model.cuda()

    # On récupère les données
    print('Récupération des données')
    train, test = get_dataset(BATCH_SIZE, PATH)
#    if CUDA:
#        cuda = True
#    else:
#        cuda = False
#    trainVGGfc(vgg16,train, test ,100, 0.1, cuda)


#     Extraction des features
    print('Feature extraction')
    X_train, y_train = extract_features(train, model)
    X_test, y_test = extract_features(test, model)

    # TODO Apprentissage et évaluation des SVM à faire
    print('Apprentissage des SVM')
    
    #test for different C
    acc = []
    c = [0.1, 0.6, 1,  5, 10]
    for value in c:
        svm = LinearSVC(C=value)
        svm.fit(X_train, y_train)
        accuracy = svm.score(X_test, y_test)
        print('C= ', value, 'accuracy= ', accuracy)
        acc.append(accuracy)

In [ ]:
# Paramètres en ligne de commande
parser = argparse.ArgumentParser()
BATCH_SIZE = 128
PATH = "15SceneData"
CUDA = True
cudnn.benchmark = True

main()

input("done")

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /tmp/torch\alexnet-owt-4df8aa71.pth
41.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

54.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

68.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Curr

Récupération des données
Feature extraction


C:\Users\Sebastien\Anaconda3\envs\Deep Learning\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


0
Batch 000/012
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
1
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
2
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
3
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
4
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
5
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
6
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
7
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 4096])
8
torch.Size([128, 3, 224, 224])
torch.Size([128, 256, 6, 6])
torch.Size([128, 9216])
torch.Size([128, 409